In [4]:
import functools
import math
import operator
import random

In [21]:
def fast_miller_rabin(n, use_probabilistic=False, tolerance=30):
    """
    Tests whether a number is prime using a deterministic version of the Miller-
    Rabin primality test. Optionally tests whether the specified number is a
    prime probabilistically up to a given tolerance using the regular version of
    the Miller-Rabin test. If the number is greater than 10^36, then all witnesses
    in the range [2, 2*log(n)*log(log(n))] are tested. However, this is conjectural
    and only heuristic evidence exists for it. To certify that a number is actually
    prime, one needs to test all witnesses in the range [2, 2*log(n)^2]. However,
    this is generally quite slow.
    Arguments:
        n (:int) - the integer to be tested
        use_probabilistic (:bool) - flag to indicate whether to use the regular
                                   version of the Miller-Rabin primality test
        tolerance (:int) - number of trials to be used to test primality
    Returns:
        True if 'n' is prime (or probably prime) and False otherwise
    References:
        - Francky from the PE Forums
        - https://miller-rabin.appspot.com/
        - https://en.wikipedia.org/wiki/Miller-Rabin_primality_test
    """
    firstPrime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]
    # Determine bases for deterministic Miller-Rabin test
    if n >= 10 ** 36:
        log_n = math.log(n)
        if not use_probabilistic:
            w = range(2, 2 * int(log_n * math.log(log_n) / math.log(2)))
        else:
            w = range(tolerance)
    elif n >= 1543267864443420616877677640751301:
        w = firstPrime[:20]
    elif n >= 564132928021909221014087501701:
        w = firstPrime[:18]
    elif n >= 59276361075595573263446330101:
        w = firstPrime[:16]
    elif n >= 6003094289670105800312596501:
        w = firstPrime[:15]
    elif n >= 3317044064679887385961981:
        w = firstPrime[:14]
    elif n >= 318665857834031151167461:
        w = firstPrime[:13]
    elif n >= 3825123056546413051:
        w = firstPrime[:12]
    # [2, 3, 5, 7, 11, 13, 17, 19, 23]
    elif n >= 341550071728321:
        w = firstPrime[:9]
    # [2, 3, 5, 7, 11, 13, 17]
    elif n >= 3474749660383:
        w = firstPrime[:7]
    elif n >= 2152302898749:
        w = firstPrime[:6]
    # [2, 3, 5, 7, 11, 13]
    elif n >= 4759123141:
        w = firstPrime[:5]
    # [2, 3, 5, 7, 11]
    elif n >= 9006403:
        w = [2, 7, 61]
    elif n >= 489997:
        # Some Fermat stuff
        if n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101:
            hn, nm1 = n >> 1, n - 1
            p = pow(2, hn, n)
            if p == 1 or p == nm1:
                p = pow(3, hn, n)
                if p == 1 or p == nm1:
                    p = pow(5, hn, n)
                    return p == 1 or p == nm1
        return False
    elif n >= 42799:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and pow(2, n - 1, n) == 1 and pow(5, n - 1, n) == 1
    elif n >= 841:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101 and n % 103 and pow(2, n - 1, n) == 1
    elif n >= 25:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23
    elif n >= 4:
        return n & 1 and n % 3
    else:
        return n > 1
    if not (n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17
            and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43
            and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73
            and n % 79 and n % 83 and n % 89):
        return False
    # Miller-Rabin
    s = 0
    d = n - 1
    while not d & 1:
        d >>= 1
        s += 1
    for k in w:
        # Pick a random witness if probabilistic
        if use_probabilistic:
            p = random.randint(2, n - 2)
        else:
            p = k
        x = pow(p, d, n)
        if x == 1:
            continue
        for _ in range(s):
            if x + 1 == n:
                break
            x = x * x % n
        else:
            return False
    return True


# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if fast_miller_rabin(x):
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return [(x, 1)]
    factors = []
    pollard_rho_rec(x, factors)
    return [(f, factors.count(f)) for f in set(factors)]

### Use brute force search for abc-hits with minor optimizations to speed up the searching process
Note: with $K = 120000$, the code takes $\approx$ 8 hours to execute, and so it can be optimized more to improve overall time complexity.

In [27]:
K = 120000

sum_c = 0
for c in range(2, K):
    c_pollard_rho = pollard_rho_prime_factorization(c)
    if all(factor[1] == 1 for factor in c_pollard_rho):
        continue
    c_prime_factors = [factor[0] for factor in c_pollard_rho]
    for a in range(1, c // 2 if c % 2 == 0 else c // 2 + 1, 2 if c % 2 == 0 else 1):
        b = c - a
        if math.gcd(a, b) != 1 or math.gcd(a, c) != 1 or math.gcd(b, c) != 1:
            continue
        unique_abc_prime_factors = [factor[0] for factor in pollard_rho_prime_factorization(a)] + [factor[0] for factor in pollard_rho_prime_factorization(b)] + c_prime_factors
        rad = functools.reduce(operator.mul, set(unique_abc_prime_factors), 1)  # Since gcd(a, b) = gcd(a, c) = gcd(b, c) = 1
        if rad >= c:
            continue
        sum_c += c
        
    if c % 1000 == 0:  # To display progress due to long execution times for large K
        print(c)
        
sum_c

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000


18407904